![1](images/TENSOR_TUNDRA_400_1.png)
    
![2](images/TENSOR_TUNDRA_400_2.png)

![3](images/TENSOR_TUNDRA_400_3.png)
<!--     
![4](images/TENSOR_TUNDRA_400_4.png) -->

In [1]:
import json
import pennylane as qml
import pennylane.numpy as np


symbols = ["H", "H", "H"]


def h3_ground_energy(bond_length):
    
    """
    Uses VQE to calculate the ground energy of the H3+ molecule with the given bond length.
    
    Args:
        - bond_length(float): The bond length of the H3+ molecule modelled as an
        equilateral triangle.
    Returns:
        - Union[float, np.tensor, np.array]: A float-like output containing the ground 
        state of the H3+ molecule with the given bond length.
    """
    


    geometry = np.array([[0.0, 0.0, 0.0], [0.5 * bond_length, 0.5 * np.sqrt(3) * bond_length, 0.0],
                     [bond_length, 0.0, 0.0]], requires_grad=True)


    hamiltonian, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=1)
    # print(qubits)
    dev = qml.device("default.qubit", wires=qubits)
    hf_state = np.array([1, 1, 0, 0, 0, 0])
    params = [np.array([0.0], requires_grad=True),np.array([0.0], requires_grad=True)]  # initial values of the circuit parameters
    args = [params] 
    def generate_circuit(mol):
        @qml.qnode(dev)
        def circuit(*args):
            qml.BasisState(hf_state, wires=[0, 1, 2, 3, 4, 5])  # Fix typo here
            qml.DoubleExcitation(args[0][0], wires=[0, 1, 2, 3])
            qml.DoubleExcitation(args[0][1], wires=[0, 1, 4, 5])
            return qml.expval(
                qml.qchem.molecular_hamiltonian(mol.symbols, mol.coordinates, charge=mol.charge, args=args[1:])[0])

        return circuit
    
    for n in range(11):
        mol = qml.qchem.Molecule(symbols, geometry, charge=1)
        args = [params, geometry]  # initial values of the differentiable parameters

    # compute gradients with respect to the circuit parameters and update the parameters
        g_params = qml.grad(generate_circuit(mol), argnum=0)(*args)
        params = params - 0.5 * g_params[0]


   
    return generate_circuit(mol)(*args)
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    outs = h3_ground_energy(ins)
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(solution_output,expected_output, atol = 1e-4), "Not the correct ground energy"
